# Data Cleaning Notebook

Import the necessary modules and download the stopword list from the NLTK source. Define a stopword list by limiting the downloaded stopwords to English words.

In [1]:
import pandas as pd
from tqdm import tqdm
import os
import re
from pathlib import Path

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Choose an article type to create sparklines for. Possible choices are 'research-article' and 'book-review'.

In [2]:
article_type = 'research-article'

Declare a variable to hold the path to the notebook and a variable to hold the path to the unigram data.

In [3]:
file_dir = Path(os.getcwd()).resolve()
unigram_dir = file_dir / 'ngram1'

The following code block defines a function which collects all the unigram data for the given article type and an optionally-given list of article IDs. It returns a dataframe with the unigrams as one column and their corresponding counts as the second column. The dataframe is sorted by unigram count in descending order.

In [4]:
def make_unigram_table(atype, a_ids=None):
    """Makes a dataframe by concatenating all unigrams from
        articles specified by a_ids.

    Arguments:
        atype {String} -- the article type to source unigrams
                          from: 'book-review', 'research-article',
                          or 'both'.

    Keyword Arguments:
        a_id {List} -- List of article IDs to include, by default
                       all articles are included. (default: {None})
    """
    if atype == "both":
        raise ValueError("Feature not yet supported!\n")
    curr_path = unigram_dir / atype
    dfs = []
    for f in tqdm(curr_path.iterdir()):
        f_id = re.search(f"{atype}/(.*)-n", str(f)).group(1)
        if a_ids is not None:
            if f_id in a_ids:
                # print("Match!")  # for debugging
                unigrams = pd.read_csv(f, sep='\t', names=["word", "count"])
                # print(f"Just read {f_id}")  # for debugging
                dfs.append(unigrams)
            else:
                # print(f"{f_id} not in a_ids list")
                continue
        else:
            unigrams = pd.read_csv(f, sep='\t', names=["word", "count"])
            # print(f"Just read {f_id}")  # for debugging
            dfs.append(unigrams)
    print(f"\nCollected {len(dfs)} articles\n")
    if not dfs:
        raise ValueError("Unable to collect dataframes")
    concat_df = pd.concat(dfs)
    summed_df = concat_df.groupby(["word"]).sum().reset_index()
    sorted_df = summed_df.sort_values(by=["count"], ascending=False)
    return sorted_df

Call the function with the given article type and inspect the resulting dataframe of unigrams and their counts.

In [5]:
unigrams = make_unigram_table(article_type)
print(f"The unigram dataframe has size: {unigrams.shape}")
unigrams.head()

392it [00:01, 372.74it/s]

Collected 392 articles

The unigram dataframe has size: (209491, 2)


,word,count
153833,s,35062
3270,1,28182
76686,his,26923
114170,n,23915
78330,i,22514


Removing any unigrams with counts below a predetermined threshold can result in quicker processing and more succint analysis. The following function does just that.

In [6]:
def drop_counts(df, threshold):
    """Removes rows from an ngram table with counts fewer than threshold.

    Arguments:
        df {Pandas dataframe} -- ngram table with columns "word" and "count"
        threshold {Integer} -- minimum count to keep rows
    """
    dropped_df = df[df['count'] >= threshold]
    return dropped_df

After filtering unigrams by their counts with a threshold of 10, the number of unigrams decreases from ~200,000 to -25,000.

In [7]:
truncated_unigrams = drop_counts(unigrams, 10)
print(f"The truncated unigrams dataframe has size: {truncated_unigrams.shape}")
truncated_unigrams.head()

The truncated unigrams dataframe has size: (24863, 2)


,word,count
153833,s,35062
3270,1,28182
76686,his,26923
114170,n,23915
78330,i,22514


The following function lemmatizes the unigrams in the table. Lemmatization is the process of aiming to remove inflectional endings of a word and to return the base or dictionary form, which is known as the lemma. For example, "car", "cars", "car's", and "cars'" all map to the base for of 'car'. This reduces noise and allows for more accurate analysis of any underlying patterns.

In [8]:
def lemmatize_table(df):
    """Lemmatizes the words in the word column of an ngram table. Sums
       frequencies of words that map to the same lemma.

    Arguments:
        df {Pandas dataframe} -- ngram table with columns 'word' and 'count'.
    """
    if df is None:
        print("The passed object is of None type")
        return "Error"
    lemmatizer = WordNetLemmatizer()

    def lemmatize_word(word):
        """Uses lemmatization from spacy to map words to lemmas.
           (Could do with some tweaking to better group lemmas).

        Arguments:
            word {String} -- lowercase word to lemmatize

        Returns:
            String -- lemmatized word
        """
        return lemmatizer.lemmatize(word)
    df_lemma = df.copy()
    df_lemma['word'] = df['word'].map(lemmatize_word, na_action="ignore")
    summed_df = df_lemma.groupby(["word"]).sum().reset_index()
    sorted_df = summed_df.sort_values(by=["count"], ascending=False)
    return sorted_df

Call the function on the dataframe of truncated unigrams.

In [12]:
lemmatized_unigrams = lemmatize_table(truncated_unigrams)
print(f"The lemmatized unigram dataframe has size: {lemmatized_unigrams.shape}")
lemmatized_unigrams.head()

The lemmatized unigram dataframe has size: (22739, 2)


,word,count
17225,s,35250
85,1,28275
9516,his,26923
13307,n,24388
9750,i,22514


The following block defines two functions. The first, **is_year** is a helper function that makes an educated guess as to whether a string composed entirely of numbers represents a year or not. It is valid for years between 1000 and 2999 CE.

The second function, **remove_numerals** uses **is_year* and removes unigrams from a unigram table that are composed partially or entirely of numerals (depending on a keyword argument which is true by default). The function attemps to retain numerals that most likely represent years, as these are meaningful in analytic contexts.

In [13]:
def is_year(string):
    """Returns a boolean value if the input string likely represents a year.
       Only valid for years 1000-2999.
    Arguments:
        string {String} -- candidate string
    """
    return bool(re.match("(1|2)\d{3}", string))


def remove_numerals(df, remove_mixed_strings=True):
    """Removes rows from an ngram table with words that are numerals. This
       does not include 4-digit numbers which are interpreted as years.

    Arguments:
        df {Pandas dataframe} -- A dataframe of with columns 'word', 'count'.

    Keyword Arguments:
        remove_mixed_strings {bool} -- Whether to remove rows with words that
        are mixtures of numerals and letters. (default: {True})
    """
    no_numerals_df = df.copy().reset_index()
    for i, row in tqdm(no_numerals_df.iterrows()):
        word = row['word']
        if remove_mixed_strings:
            if any([c.isnumeric() for c in word]) and \
               not is_year(word):
                no_numerals_df.drop(i, axis=0, inplace=True)
        else:
            if word.isnumeric() and len(word) != 4:
                no_numerals_df.drop(i, axis=0, inplace=True)
    return no_numerals_df

Calling the function on the lemmatized unigrams returns the following dataframe:

In [14]:
no_nums_df = remove_numerals(lemmatized_unigrams)
print(f"\nThe unigram dataframe with numerals removed has size: {no_nums_df.shape}")
no_nums_df.head()

22739it [00:07, 3122.87it/s]
The unigram dataframe with numerals removed has size: (20992, 3)



,index,word,count
0,17225,s,35250
2,9516,his,26923
3,13307,n,24388
4,9750,i,22514
5,9286,he,20488


If nedded - use the following function to create a custom stop list for eventual removal.

In [11]:
def make_custom_stopword_list(df):
    """Creates and saves a custom stopword list as a plain text file in the data directory.

    Arguments:
        df {Pandas Dataframe} -- A dataframe of with columns 'word', 'count', 'stopword'
        which is a table of ngram frequencies that has been manually labeled with an 'x'
        to indicate inclusion. Words marked with 'r' have already been reviewed and will
        not be added to the stop list. All other words are added to the stop list.
    """
    stopwords = []
    for i, row in tqdm(df.iterrows(), desc="Creating stopwords\n"):
        stopword_status = row['stopword']
        if stopword_status == 'r': # if 'r', word has already been reviewed, so ignore
            continue
        if stopword_status == 'x': # if 'x', word is marked for inclusion, so ignore
            df.loc[i,'stopword'] = 'r' # update 'x' to 'r'
        if stopword_status == "unchecked": # if 'unchecked', wait for manual review
            continue
        else:
            stopwords.append(row['word'])
    with open("custom_stopwords.txt", "w+") as filehandle:
        for word in stopwords:
            filehandle.write(f"{word}\n")

The following function removes stopwords from a unigram table: either those in the NLTK stopword list or those supplied from a custom list. Crafting a custom stopword list is time-consuming but often worthwhile; the NLTK list is not adapted for the specfic vocabulary of any one field.

In [15]:
def remove_stopwords(df, include_custom=False):
    """Removes rows from an ngram table with words
       in a custom stopword list or in NLTK list.

    Arguments:
        df {Pandas dataframe} -- ngram table with columns "word" and "stopword"
    """
    custom_stop = set()
    if include_custom:
        with open("custom_stopwords.txt") as f:
            custom_stop = f.readlines()
    custom_stop = [w.strip() for w in custom_stop]
    no_stops_df = df.copy()
    for i, row in tqdm(no_stops_df.iterrows()):
        word = row['word']
        if include_custom:
            if word in nltk_stopwords or word in custom_stop:
                no_stops_df.drop(i, axis=0, inplace=True)
        else:
            if word in nltk_stopwords:
                no_stops_df.drop(i, axis=0, inplace=True)
    return no_stops_df

Calling the function on the unigrams with numerals removed returns the following dataframe:

In [16]:
no_stops_df = remove_stopwords(no_nums_df)
print(f"The unigram dataframe with stopwords removed has size: {no_stops_df.shape}")
no_stops_df.head()

20992it [00:02, 7435.10it/s]The unigram dataframe with stopwords removed has size: (20874, 3)



,index,word,count
3,13307,n,24388
7,10909,jewish,17390
9,8993,ha,15356
10,17571,see,15304
11,14554,p,13117


This is the end of the data cleaning process. Save the final dataframe to a CSV file:

In [17]:
no_stops_df.to_csv("articles_cleaned.csv", index=False)